# Final Project Term Paper
Faculty for Environment and Natural Resources
Module „Supply chain modelling, indicators and responsibility “, Sommer term 2023

## Step 2: Map COICOP categories to EXIOBASE products

### Part 1

In [1]:
import pandas as pd
import numpy as np
import os

ct = pd.read_excel("CT_coicop_exiobase_done.xlsx")

# extract raw data that we need
ct_raw = ct.iloc[2:, 1:]
ct_raw.head()

,AT,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 9791,Unnamed: 9792,Unnamed: 9793,Unnamed: 9794,Unnamed: 9795,Unnamed: 9796,Unnamed: 9797,Unnamed: 9798,Unnamed: 9799,Unnamed: 9800
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,1,1,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Question: What is the shape of the CT? Are there more COICOP categories than EXIOBASE products or vice versa? What does this mean for the mapping?

In [2]:
print("Shape:", ct_raw.shape)

Shape: (47, 9800)


This means that there are 47 COICOP categories that correspond to 9800 EXIOBASE products. One category is expected to have many product mappings, making it quite "general".

In [3]:
# import exio partialy (for speed increase)
exio3_folder = "/users/dannyeel/documents/pymrio"
# exio3 = pymrio.parse_exiobase3(os.path.join(exio3_folder, "IOT_2015_pxp.zip")) # alternatively
Y = pd.read_table(
    os.path.join(exio3_folder, "IOT_2015_pxp/Y.txt"), header=[0, 1], index_col=[0, 1]
)

If we run `exio3.get_regions()` we will get:

    Index(['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR',
       'HR', 'HU', 'IE', 'IT', 'LT', 'LU', 'LV', 'MT', 'NL', 'PL', 'PT', 'RO',
       'SE', 'SI', 'SK', 'GB', 'US', 'JP', 'CN', 'CA', 'KR', 'BR', 'IN', 'MX',
       'RU', 'AU', 'CH', 'TR', 'TW', 'NO', 'ID', 'ZA', 'WA', 'WL', 'WE', 'WF',
       'WM'],
      dtype='object', name='region')

Seems like we need to use 'GB' instead of 'UK' as I initially expected. While UK, Great Britain and England are not the same thing, we'll see that the final sum (between COICOP and EXIO databases) checks out. We will assume this to be sufficient for the purposes of this paper.

In [4]:
Y_GB = Y.loc[:, ("GB", "Final consumption expenditure by households")]
Y_GB

region  sector                                           
AT      Paddy rice                                             0.000000
        Wheat                                                  0.000000
        Cereal grains nec                                      1.342980
        Vegetables, fruit, nuts                                8.199273
        Oil seeds                                              0.871978
                                                                ...    
WM      Membership organisation services n.e.c. (91)           0.121621
        Recreational, cultural and sporting services (92)     22.503670
        Other services (93)                                    6.105621
        Private households with employed persons (95)        984.609084
        Extra-territorial organizations and bodies             0.000000
Name: (GB, Final consumption expenditure by households), Length: 9800, dtype: float64

In [5]:
# define matrix multiplication factors
foo = ct_raw.to_numpy()
bar = Y_GB.to_numpy()
print(foo.shape, bar.shape)

(47, 9800) (9800,)


---
#### ___Side note___:

Initially I wanted to perform matrix multiplication with explicit diagonalization as follows:
    
        foo = ct_raw.to_numpy()
        bar = Y_GB.to_numpy()
        np.matmul(foo, np.diag(bar))

However, that took 4m 32.2s to calculate on MacBook Air M2 (2022). Running the following code was instant:

        foo = ct_raw.to_numpy()
        bar = Y_GB.to_numpy()
        np.multiply(foo, bar)

What's interesting is that both methods give exactly the same result and I have absolutely no idea why.

---

In [6]:
ct_filled = pd.DataFrame(np.multiply(foo, bar))

ct_filled.columns = Y_GB.index
ct_filled.index = ct.iloc[2:, 0]
ct_filled.index.name = "coicop"

ct_filled.head()

region         AT                                                            \
sector Paddy rice Wheat Cereal grains nec Vegetables, fruit, nuts Oil seeds   
coicop                                                                        
c01.1         0.0   0.0           1.34298                8.199273  0.871978   
c01.2         0.0   0.0           1.34298                8.199273  0.871978   
c02.1         0.0   0.0               0.0                     0.0       0.0   
c02.2         0.0   0.0               0.0                     0.0       0.0   
c02.3         0.0   0.0           1.34298                     0.0  0.871978   

region                                                                  ...  \
sector Sugar cane, sugar beet Plant-based fibers Crops nec Cattle Pigs  ...   
coicop                                                                  ...   
c01.1                     0.0                0.0  0.162602    0.0  0.0  ...   
c01.2                     0.0                0.0  0.162602    0.0  0.0  ...   
c02.1                     0.0                0.0       0.0    0.0  0.0  ...   
c02.2                     0.0                0.0       0.0    0.0  0.0  ...   
c02.3                     0.0                0.0  0.162602    0.0  0.0  ...   

region                            WM                                        \
sector Paper for treatment: landfill Plastic waste for treatment: landfill   
coicop                                                                       
c01.1                            0.0                                   0.0   
c01.2                            0.0                                   0.0   
c02.1                            0.0                                   0.0   
c02.2                            0.0                                   0.0   
c02.3                            0.0                                   0.0   

region                                                      \
sector Inert/metal/hazardous waste for treatment: landfill   
coicop                                                       
c01.1                                                 0.0    
c01.2                                                 0.0    
c02.1                                                 0.0    
c02.2                                                 0.0    
c02.3                                                 0.0    

region                                         \
sector Textiles waste for treatment: landfill   
coicop                                          
c01.1                                     0.0   
c01.2                                     0.0   
c02.1                                     0.0   
c02.2                                     0.0   
c02.3                                     0.0   

region                                     \
sector Wood waste for treatment: landfill   
coicop                                      
c01.1                                 0.0   
c01.2                                 0.0   
c02.1                                 0.0   
c02.2                                 0.0   
c02.3                                 0.0   

region                                               \
sector Membership organisation services n.e.c. (91)   
coicop                                                
c01.1                                           0.0   
c01.2                                           0.0   
c02.1                                           0.0   
c02.2                                           0.0   
c02.3                                           0.0   

region                                                                        \
sector Recreational, cultural and sporting services (92) Other services (93)   
coicop                                                                         
c01.1                                                0.0                 0.0   
c01.2                                                0.0                 0.0   
c02.1            

In [7]:
# normalize rows and replace NaNs with zeros
ct_filled = ct_filled.div(ct_filled.sum(axis=1), axis=0)
ct_filled.fillna(0, inplace=True)

ct_filled.head()

region         AT                                                            \
sector Paddy rice Wheat Cereal grains nec Vegetables, fruit, nuts Oil seeds   
coicop                                                                        
c01.1         0.0   0.0           0.00001                0.000062  0.000007   
c01.2         0.0   0.0          0.000018                0.000112  0.000012   
c02.1         0.0   0.0               0.0                     0.0       0.0   
c02.2         0.0   0.0               0.0                     0.0       0.0   
c02.3         0.0   0.0          0.000021                     0.0  0.000014   

region                                                                  ...  \
sector Sugar cane, sugar beet Plant-based fibers Crops nec Cattle Pigs  ...   
coicop                                                                  ...   
c01.1                     0.0                0.0  0.000001    0.0  0.0  ...   
c01.2                     0.0                0.0  0.000002    0.0  0.0  ...   
c02.1                     0.0                0.0       0.0    0.0  0.0  ...   
c02.2                     0.0                0.0       0.0    0.0  0.0  ...   
c02.3                     0.0                0.0  0.000003    0.0  0.0  ...   

region                            WM                                        \
sector Paper for treatment: landfill Plastic waste for treatment: landfill   
coicop                                                                       
c01.1                            0.0                                   0.0   
c01.2                            0.0                                   0.0   
c02.1                            0.0                                   0.0   
c02.2                            0.0                                   0.0   
c02.3                            0.0                                   0.0   

region                                                      \
sector Inert/metal/hazardous waste for treatment: landfill   
coicop                                                       
c01.1                                                 0.0    
c01.2                                                 0.0    
c02.1                                                 0.0    
c02.2                                                 0.0    
c02.3                                                 0.0    

region                                         \
sector Textiles waste for treatment: landfill   
coicop                                          
c01.1                                     0.0   
c01.2                                     0.0   
c02.1                                     0.0   
c02.2                                     0.0   
c02.3                                     0.0   

region                                     \
sector Wood waste for treatment: landfill   
coicop                                      
c01.1                                 0.0   
c01.2                                 0.0   
c02.1                                 0.0   
c02.2                                 0.0   
c02.3                                 0.0   

region                                               \
sector Membership organisation services n.e.c. (91)   
coicop                                                
c01.1                                           0.0   
c01.2                                           0.0   
c02.1                                           0.0   
c02.2                                           0.0   
c02.3                                           0.0   

region                                                                        \
sector Recreational, cultural and sporting services (92) Other services (93)   
coicop                                                                         
c01.1                                                0.0                 0.0   
c01.2                                                0.0                 0.0   
c02.1            

#### Question: Why is it important that each row of the filled CT sums to one?

Because we are looking for the share of the expenditure per COICOP category, we need to normalize the multiplicand. If this was not the case, we'd get sector values magnitudes greater than their actual values, since we'd simply be taking the entire quintile values instead of their respective shares.

### Part 2

In [8]:
abs_exp = pd.read_excel("abs_exp.xlsx")

# fix import inconsistencies
abs_exp.index = abs_exp.coicop
abs_exp.drop(columns=["coicop"], inplace=True)
abs_exp.head()

,QUINTILE1,QUINTILE2,QUINTILE3,QUINTILE4,QUINTILE5
coicop,,,,,
CP011,1657.260,2207.387,2822.085,3267.395,4062.586
CP012,147.015,191.191,260.865,289.150,345.752
CP021,213.840,278.096,379.440,491.555,734.723
CP022,187.110,208.572,237.150,173.490,172.876
CP023,0.000,0.000,0.000,0.000,0.000


In [9]:
foo = ct_filled.copy(deep=True).to_numpy().transpose()
bar = abs_exp.to_numpy()

quint_fin = pd.DataFrame(
    np.matmul(foo, bar), index=ct_filled.columns, columns=abs_exp.columns
)
quint_fin

QUINTILE1 QUINTILE2  \
region sector                                                                  
AT     Paddy rice                                              0.0       0.0   
       Wheat                                                   0.0       0.0   
       Cereal grains nec                                   0.02602  0.034149   
       Vegetables, fruit, nuts                            0.142641  0.189743   
       Oil seeds                                          0.016894  0.022173   
...                                                            ...       ...   
WM     Membership organisation services n.e.c. (91)            0.0       0.0   
       Recreational, cultural and sporting services (92)  0.106381   0.17471   
       Other services (93)                                0.063468  0.112167   
       Private households with employed persons (95)      0.894399  1.246236   
       Extra-territorial organizations and bodies              0.0       0.0   

                                                         QUINTILE3 QUINTILE4  \
region sector                                                                  
AT     Paddy rice                                              0.0       0.0   
       Wheat                                                   0.0       0.0   
       Cereal grains nec                                  0.046767  0.051103   
       Vegetables, fruit, nuts                            0.243957  0.280809   
       Oil seeds                                          0.030365  0.033181   
...                                                            ...       ...   
WM     Membership organisation services n.e.c. (91)            0.0       0.0   
       Recreational, cultural and sporting services (92)  0.237164  0.335445   
       Other services (93)                                0.158068  0.209141   
       Private households with employed persons (95)      1.473671  1.520373   
       Extra-territorial organizations and bodies              0.0       0.0   

                                                         QUINTILE5  
region sector                                                       
AT     Paddy rice                                              0.0  
       Wheat                                                   0.0  
       Cereal grains nec                                  0.063173  
       Vegetables, fruit, nuts                             0.35655  
       Oil seeds                                          0.041017  
...                                                            ...  
WM     Membership organisation services n.e.c. (91)            0.0  
       Recreational, cultural and sporting services (92)  0.501994  
       Other services (93)                                0.328336  
       Private households with employed persons (95)      2.685667  
       Extra-territorial organizations and bodies              0.0  

[9800 rows x 5 columns]

In [10]:
quint_fin.sum()

QUINTILE1    13351.635
QUINTILE2    17346.238
QUINTILE3    23738.715
QUINTILE4     28857.17
QUINTILE5      43219.0
dtype: object

In [11]:
abs_exp.sum()

QUINTILE1    13351.635
QUINTILE2    17346.238
QUINTILE3    23738.715
QUINTILE4    28857.170
QUINTILE5    43219.000
dtype: float64

In [12]:
print("Performing dataframe accuracy comparison!")

# conditional loop parameters
sig_fig = 3
max_iter = 50
found = False
while not found or sig_fig > max_iter:  # loop until divergence or max iteration
    # sum lists and round values, then compare
    match = all(
        np.round(list(quint_fin.sum().values), sig_fig)
        == abs_exp.sum().values.round(sig_fig)
    )
    if match is False:
        print("Divergence at {0} significant figures.".format(sig_fig))
        found = True
    else:
        print("Accurate to {0} significant figures...".format(sig_fig))
        sig_fig += 1

Performing dataframe accuracy comparison!
Accurate to 3 significant figures...
Accurate to 4 significant figures...
Accurate to 5 significant figures...
Accurate to 6 significant figures...
Accurate to 7 significant figures...
Accurate to 8 significant figures...
Accurate to 9 significant figures...
Divergence at 10 significant figures.


This is a surprisingly accurate result!

#### Question: What EXIOBASE sector-region pair shows the highest expenditure for each different group from your cross-sectional variable? If the category differs between groups, why do you think this is the case?

In [13]:
foo = []
col_names = []
for group in quint_fin.columns:
    max_val = quint_fin[group].max()
    bar = quint_fin[group][quint_fin[group] == max_val]
    col_names.append(bar.name)
    foo.append(bar)

df = pd.DataFrame(pd.concat(foo), columns=["Max val"])
df["Group"] = col_names
df

Max val      Group
region sector                                           
GB     Real estate services (70)  3139.365963  QUINTILE1
       Real estate services (70)  3329.938602  QUINTILE2
       Real estate services (70)  3619.647089  QUINTILE3
       Real estate services (70)  3993.121563  QUINTILE4
       Real estate services (70)  5066.312625  QUINTILE5

We observe that the max values between different groups all share the same region and sector pair, namely: GB, Real estate services (70).

In [14]:
# export to Excel
if os.path.exists("quint_fin.xlsx") is False:
    quint_fin.to_excel("quint_fin.xlsx")
    print("File exported successfully.")
else:
    print("File already exported. Delete & rerun cell to update.")

File already exported. Delete & rerun cell to update.
